In [16]:
training_Data = [ 
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Orange'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'cherry'],
    ['Blue', 2, 'BlueBerry'],

]
print(training_Data)

[['Green', 3, 'Mango'], ['Yellow', 3, 'Orange'], ['Red', 1, 'Grape'], ['Red', 1, 'cherry'], ['Blue', 2, 'BlueBerry']]


In [17]:
# Add Columns names

header = ["color", "diameter", "label"]

In [19]:
def unique_vals(rows,col):
    """ Find the unique values for a column in Dataset"""
    return set([row(col) for row in rows])

def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        
        if label not in counts:
            counts[label] = 1
        else:
            counts[label] += 1
    return counts


def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

class Question:
    def __init__(self,column,value):
        self.column = column
        self.value = value
    def match(self,row):
        val = row[self.column]
        
        
        
        if is_numeric(val):
            return val >= self.value    # True/False
        else:
            return val == self.value     # True/False
    def __repr__(self):
        condition = '=='
        if is_numeric(self.value):
            condition = '>='
            
        return "is %s %s %s?" % (header[self.column],condition, str(self.value))
    
def partition(rows, question):
    true_rows, false_rows = [] , []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
           
    return true_rows, false_rows


def gini(rows):
    """ calculate the gini impurity ofor a list of rows"""
    counts = class_counts(rows)
    
    impurity = 1
    for lbl  in counts:
        
        prob_of_lbl = counts[lbl]/float(len(rows))
        
        impurity -= prob_of_lbl**2
    
    return impurity

def info_gain(left,right,current_uncertainity):
    """ information of the starting node minus the weightened impurity of the two child nodes"""
    
    p = float(len(left))/(len(left)  + len(right))
    
    d = (current_uncertainity) - p*gini(left) - ( 1 - p )*gini(right)
    
    return d
def find_best_split(rows):
    best_gain= 0 
    best_question = None
    
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) -1
  
    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col,val)
            
            true_rows,false_rows = partition(rows,question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            
            if gain >= 0:
                if gain >= best_gain:
                    best_gain, best_question = gain, question
            else:
                if -(gain) >= best_gain:
                    best_gain, best_question = -(gain), question
                
    return best_gain, best_question

class leaf:
    """ a leaf node classifies data"""
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
    
class Decision_Node:
    """ a decision node asks a question"""
    def __init__( self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
        
def build_tree(rows):
    """builds the tree"""
    
    gain, question = find_best_split(rows)
             
    if gain == 0:
        
        return leaf(rows)
    true_rows, false_rows = partition(rows,question) 
    
    true_branch = build_tree(true_rows)
   
    false_branch = build_tree(false_rows)
    
    
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing="  "):
    """ worlds most elegant tree printing functio"""
    if isinstance(node,leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    print (spacing + str(node.question)  )
    
    print(spacing + '---> True:')
    print_tree(node.true_branch, spacing + "       ")
    
    print(spacing +' ---> False:')
    print_tree(node.false_branch, spacing + "      ")
    
def classify(row,node):
    # Base Case : we have reached a leaf
    if isinstance(node,leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    
    
def print_leaf(counts):
    """ print the predictions of a leaf"""
    
    total = (sum(counts.values()))*1.0
    probs = {}
    if total != 0:
        for lbl in counts.keys():
            probs[lbl] = str(int((counts[lbl]/total)*100 )) + "%"
    return probs



if __name__ == '__main__':
    my_tree = build_tree(training_Data)
    print_tree(my_tree)
    
    # Evaluate
    
    testing_data =  [ 
        ['Yellow', 5, 'Lemon'],
        ['Yellow',5, 'Lemon'],
        ['Red', 5, 'Lemon'],
        ['Blue', 2, 'BlueBerry'],
        ['Yellow',5, 'Lemon'],
    ]
    for row in testing_data:
        #print(classify(row,my_tree))
        
        
        
        print("Actual : %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))
    
    
    

  is diameter >= 3?
  ---> True:
         is color == Green?
         ---> True:
                Predict {'Mango': 1}
          ---> False:
               Predict {'Orange': 1}
   ---> False:
        is color == Red?
        ---> True:
               Predict {'Grape': 1, 'cherry': 1}
         ---> False:
              Predict {'BlueBerry': 1}
Actual : Lemon. Predicted: {'Orange': '100%'}
Actual : Lemon. Predicted: {'Orange': '100%'}
Actual : Lemon. Predicted: {'Orange': '100%'}
Actual : BlueBerry. Predicted: {'BlueBerry': '100%'}
Actual : Lemon. Predicted: {'Orange': '100%'}
